In [13]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.tree import export_graphviz
from tensorflow.python import keras
print(tf.__version__)

2.0.0


In [2]:
TRAIN_DIR = "./input/train"
TEST_DIR = "./input/test"

TRAIN_SIZE = len([name for name in os.listdir(TRAIN_DIR)])
TEST_SIZE = len([name for name in os.listdir(TEST_DIR)])
print("Number of training images:", TRAIN_SIZE)
print("Number of test images:", TEST_SIZE)

VALID_FRACTION = 0.2
BATCH_SIZE = 100
EPOCHS = 14

IMAGE_WIDTH = IMAGE_HEIGHT = 128

Number of training images: 6101
Number of test images: 1101


In [3]:
# creating df with train labels
train_filenames = os.listdir(TRAIN_DIR)
train_labels = []
for filename in train_filenames:
    label = filename.split('__')[0]
    train_labels.append(label)

train_df = pd.DataFrame({
    'id': train_filenames,
    'label': train_labels
})

In [4]:
# splitting to train & valid
train_df, valid_df = train_test_split(train_df, test_size=VALID_FRACTION)

In [5]:
#augmentation settings, for now just normalizing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(    
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255.,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

In [6]:
# not doing any data augmentation on validation test set
valid_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

In [7]:
# creating train and valid generators (not using valid_split to avoid doing data augmentation on validation set)
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    TRAIN_DIR, 
    x_col='id',
    y_col='label',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 4880 validated image filenames belonging to 100 classes.


In [8]:
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df, 
    TRAIN_DIR, 
    x_col='id',
    y_col='label',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 1220 validated image filenames belonging to 100 classes.


C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 1 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [9]:
model = tf.keras.models.Sequential([
    # the images were resized by ImageDataGenerator 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # since we have only 2 classes to predict we can use 1 neuron and sigmoid
    tf.keras.layers.Dense(100, activation='sigmoid')  
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       1

In [21]:
opti=tf.keras.optimizers.Adam()

In [22]:
model.compile(optimizer=opti,
    loss='categorical_crossentropy',
    metrics = ['accuracy'])

In [23]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
    mode='min',
    restore_best_weights=True, 
    verbose=1,
    patience=5)

In [82]:
#Create model image
#tf.keras.utils.plot_model(model)
#round(TRAIN_SIZE*(1.-VALID_FRACTION)/BATCH_SIZE)
#round(TRAIN_SIZE*VALID_FRACTION/BATCH_SIZE)

In [ ]:
%%time

# training
history = model.fit_generator(train_generator,
    validation_data=valid_generator,
    steps_per_epoch=300,
    validation_steps=300,
    epochs=EPOCHS,
    callbacks=[es],
    verbose=1)

Epoch 1/14
300/300 [==============================] - 1598s 5s/step - loss: 1.8399 - accuracy: 0.4337 - val_loss: 0.5714 - val_accuracy: 0.8097
Epoch 2/14
300/300 [==============================] - 1693s 6s/step - loss: 0.5718 - accuracy: 0.7994 - val_loss: 0.2091 - val_accuracy: 0.9343
Epoch 3/14
300/300 [==============================] - 1717s 6s/step - loss: 0.3113 - accuracy: 0.8883 - val_loss: 0.0684 - val_accuracy: 0.9779
Epoch 4/14
300/300 [==============================] - 1634s 5s/step - loss: 0.2571 - accuracy: 0.9098 - val_loss: 0.0888 - val_accuracy: 0.9640
Epoch 5/14
138/300 [============>.................] - ETA: 11:34 - loss: nan - accuracy: 0.3924